In [1]:
import dynamic_traffic_signal
import importlib

In [2]:
a = dynamic_traffic_signal.DynamicTrafficSignal('Testing_class_ijipura_small')

In [3]:
minlat="12.9380131671"
minlon="77.6319900345" 
maxlat="12.939314082" 
maxlon="77.6337735827"

In [4]:
a.get_map(minlon, minlat , maxlon,  maxlat)

In [5]:
diction

0

In [9]:
len(diction)

1

In [10]:
for i in range(len(diction)):
    print(" ".join(diction[i]['nodes']))

315208342 315208341


In [11]:
with open("test_node.xml", "w") as routes:
    print("<nodes>", file=routes)
    for i in range(len(diction)):
        print('   <join nodes="{}"/>'.format(" ".join(diction[i]['nodes'])), file=routes)

    print("</nodes>", file=routes)

In [28]:
from geopy.distance import great_circle
max_distance = 20
for i in net.edges:
    distance = great_circle((net.nodes[i[0]]['lat'], net.nodes[i[0]]['long']),
                           (net.nodes[i[1]]['lat'], net.nodes[i[1]]['long'])).meters
    if distance > max_distance:
        net.remove_edge(*i) 

In [29]:
a.get_raw_traffic_light_nodes()

{'61748555': {'lat': '12.9516788',
  'long': '77.6214774',
  'junction_name': 'None'},
 '260729391': {'lat': '12.9372613',
  'long': '77.6269775',
  'junction_name': 'Sony World Signal'},
 '263227076': {'lat': '12.9298524',
  'long': '77.6331784',
  'junction_name': 'None'},
 '309592695': {'lat': '12.9351857',
  'long': '77.6244386',
  'junction_name': 'None'},
 '315208341': {'lat': '12.9385881',
  'long': '77.6326767',
  'junction_name': 'None'},
 '315208342': {'lat': '12.9384851',
  'long': '77.6327639',
  'junction_name': 'Ejipur Signal'}}

In [7]:
net.edges

EdgeView([('315208341', '315208342')])

In [5]:
import os

In [9]:
os.path.realpath('..')

'/home/utkarsh/Study/Quantela'

In [10]:
os.path.realpath('/home/utkarsh/Study/Quantela')

'/home/utkarsh/Study/Quantela'